# Import

In [41]:
import tensorflow as tf
import requests
import json
import base64
import pandas as pd

# Testing prediction

Pertama, load dummy data yang akan dilakukan prediksi

In [23]:
data = pd.read_csv('dummy_data/testing.csv')
data

,OverTime,MaritalStatus,TotalWorkingYears,Age,MonthlyIncome,JobRole,Department
0,No,Married,15,37,4777,Healthcare Representative,Research & Development
1,Yes,Married,18,51,2461,Research Scientist,Research & Development


In [26]:
# Mengambil input pertama untuk prediksi
inputs = data.iloc[0, :7].to_dict()  # Mengambil fitur hingga kolom ke-7
inputs

{'OverTime': 'No',
 'MaritalStatus': 'Married',
 'TotalWorkingYears': 15,
 'Age': 37,
 'MonthlyIncome': 4777,
 'JobRole': 'Healthcare Representative',
 'Department': 'Research & Development'}

Lalu, menyiapkan input JSON yang sesuai dengan skema model

In [27]:
# Fungsi untuk menyiapkan data input
def prepare_json(inputs: dict):
    # Mapping fitur ke TensorFlow Feature
    feature_mapping = {
        "OverTime": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs['OverTime'].encode()])),
        "MaritalStatus": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs['MaritalStatus'].encode()])),
        "TotalWorkingYears": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['TotalWorkingYears'])])),
        "Age": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['Age'])])),
        "MonthlyIncome": tf.train.Feature(int64_list=tf.train.Int64List(value=[int(inputs['MonthlyIncome'])])),
        "JobRole": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs['JobRole'].encode()])),
        "Department": tf.train.Feature(bytes_list=tf.train.BytesList(value=[inputs['Department'].encode()])),
    }

    # Membuat TensorFlow Example
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_mapping)
    ).SerializeToString()

    # Melakukan encode Example ke base64
    result = {
        "examples": {
            "b64": base64.b64encode(example).decode()
        }
    }

    # Mengembalikan data dalam format JSON
    return json.dumps({
        "signature_name": "serving_default",
        "instances": [result],
    })


Membuat fungsi untuk melakukan prediksi

In [28]:
# Fungsi untuk memprediksi data
def predict(inputs):
    json_data = prepare_json(inputs)

    # URL endpoint REST API model
    endpoint = "https://tfx-mlops-project-production.up.railway.app/v1/models/attrition-model:predict"  # Ganti URL dengan endpoint API Anda

    # Mengirim request ke endpoint
    response = requests.post(endpoint, data=json_data)

    # Periksa apakah respons berhasil (HTTP status code 200)
    if response.status_code != 200:
        return f"Error: API response status code {response.status_code}. Response: {response.text}"

    # Parsing hasil prediksi
    response_json = response.json()
    prediction = response_json.get("predictions")

    # Periksa apakah kunci "predictions" ada di respons
    if prediction is None:
        return f"Error: No 'predictions' key found in the response. Full Response: {response_json}"

    # Mengambil prediksi pertama
    prediction_value = prediction[0][0]
    if prediction_value > 0.5:
        result = "Label 1 (Attrition: Yes)"
    else:
        result = "Label 0 (Attrition: No)"
    
    return result


Prediksi dummy data no 1

In [39]:
# Predict data
inputs = data.iloc[0, :7].to_dict()
print(predict(inputs))

Label 0 (Attrition: No)


Prediksi dummy data no 2

In [42]:
# Predict data
inputs = data.iloc[1, :7].to_dict()
print(predict(inputs))


Label 1 (Attrition: Yes)
